### Importazione librerie

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle


### Caricamento file train.csv

In [33]:
file_path = "../../train.csv"
df = pd.read_csv(file_path)

### Controllo ed eliminazione valori nulli e duplicati

In [34]:
# Valori mancanti per ciascuna colonna
missing_values_per_column = df.isnull().sum()
print("\nValori mancanti nel DataFrame per ciascuna colonna:")
print(missing_values_per_column)

# Totale dei valori mancanti in tutto il DataFrame
total_missing_values = missing_values_per_column.sum()
print(f"\nTotale dei valori mancanti nel DataFrame:\n{total_missing_values}")


Valori mancanti nel DataFrame per ciascuna colonna:
Year    0
S0      0
S1      0
S2      0
S3      0
       ..
S85     0
S86     0
S87     0
S88     0
S89     0
Length: 91, dtype: int64

Totale dei valori mancanti nel DataFrame:
0


In [35]:
print("\nDuplicati nel DataFrame:")
print(df.duplicated().sum())


Duplicati nel DataFrame:
52


In [36]:
clean_df = df.dropna() 
clean_df = df.drop_duplicates()